In [1]:
import random
import ipywidgets as widgets
from IPython.display import display, clear_output

# Define the Hangman game class
class HangmanGame:
    def __init__(self, word_list, max_attempts=6):
        self.word_list = word_list
        self.max_attempts = max_attempts
        self.reset_game()

    def reset_game(self):
        self.answer = random.choice(self.word_list).lower()
        self.masked_word = '*' * len(self.answer)
        self.guessed_letters = set()
        self.attempts_left = self.max_attempts
        self.game_over = False

    def guess_letter(self, letter):
        letter = letter.lower()

        if self.game_over:
            return "Game over. Please start a new game."

        if not self._is_valid_letter(letter):
            return "❌ Invalid input. Please enter a single letter."

        if letter in self.guessed_letters:
            return f"⚠️ You've already guessed '{letter}'. Try a different letter."

        self.guessed_letters.add(letter)

        if letter in self.answer:
            self._update_masked_word(letter)
            if self.masked_word == self.answer:
                self.game_over = True
                return f"🎉 Congratulations! You won! The word was '{self.answer}'."
            return f"✅ Correct! '{letter}' is in the word.\n{self._get_game_status()}"
        else:
            self.attempts_left -= 1
            if self.attempts_left == 0:
                self.game_over = True
                return f"😢 Game over! You lost. The word was '{self.answer}'."
            return f"❌ Wrong! '{letter}' is not in the word.\n{self._get_game_status()}"

    def _is_valid_letter(self, letter):
        return len(letter) == 1 and letter.isalpha()

    def _update_masked_word(self, letter):
        new_masked = list(self.masked_word)
        for i, char in enumerate(self.answer):
            if char == letter:
                new_masked[i] = self.answer[i]
        self.masked_word = ''.join(new_masked)

    def _get_game_status(self):
        return (
            f"🔠 Word: {self.masked_word}\n"
            f"❤️ Attempts left: {self.attempts_left}\n"
            f"📝 Guessed letters: {', '.join(sorted(self.guessed_letters))}"
        )

# Function to activate the game interface in Google Colab
def play_hangman_colab():
    word_list = ["Python", "Hangman", "Programming", "Algorithm", "Developer", "Google", "Colab"]
    game = HangmanGame(word_list)

    guess_input = widgets.Text(description="Guess:")
    output = widgets.Output()
    new_game_btn = widgets.Button(description="🔄 New Game", button_style='success')

    # When user submits a guess
    def on_guess_submit(sender):
        with output:
            clear_output()
            guess = guess_input.value.strip().lower()
            guess_input.value = ''

            if not guess:
                print("Please enter a guess.")
                return

            result = game.guess_letter(guess)
            print(result)

            if game.game_over:
                print("\n💡 Game ended. Click 'New Game' to play again.")
                guess_input.disabled = True

    # When user clicks "New Game"
    def on_new_game(sender):
        with output:
            clear_output()
            game.reset_game()
            guess_input.disabled = False
            print("🎮 New game started!")
            print(game._get_game_status())

    # Bind input and button actions
    guess_input.on_submit(on_guess_submit)
    new_game_btn.on_click(on_new_game)

    # Display the game UI
    display(widgets.VBox([
        widgets.HTML("<h2>🎯 Hangman Game</h2>"),
        guess_input,
        new_game_btn,
        output
    ]))

    with output:
        print(game._get_game_status())

# Start the game
play_hangman_colab()